# Smart EDGAR: Forecasting and Calculation of Surprises

Financial KPIs can be used to drive investment decisions. So it was my goal to create a comprehensive set of KPIs across different __dimensions__ that are based on the information which can be determined from EDGAR: 
- Profitability
- Liquidity
- Efficiency
- Innovation
- Growth
- Leadership
- Surprises

In this document we demonstrate the approach on how to calculate the __Surprises__

## Setup
We install the dependencies with Maven

In [1]:
%classpath config resolver maven-public http://software.pschatzmann.ch/repository/maven-public/
%%classpath add mvn 
ch.pschatzmann:smart-edgar:1.0.2
ch.pschatzmann:jupyter-jdk-extensions:1.0.0


Added new repo: maven-public


In [2]:
import ch.pschatzmann.edgar.reporting.company._
import ch.pschatzmann.display.Displayers

Displayers.setup

true

## Comparision with Forecasted Value
The simplest approach to determine a Surprise is to calculate the forecasted value and to compare it with the effectivly reported value. 

The calculation can be done with the forecast() method. The surprisePercent() gives the difference in % of the effective value compared to the forecast value.

In [3]:
val values = new CompanyEdgarValuesDB(new CompanySelection().setTradingSymbol("AAPL"))
    .setAddTime(true)
    .setFilter(new FilterQuarterlyCumulated())
    .setParameterNames("NetIncomeLoss", "Revenues","SalesRevenueNet","EarningsPerShareBasic")
    .addFormula("Revenue","Edgar.coalesce('Revenues', 'SalesRevenueNet')")
    .addFormula("NetIncomeForecasted","Edgar.forecast('NetIncomeLoss')")
    .addFormula("RevenueForecasted","Edgar.forecast('Revenue')")
    .addFormula("EarningsPerShareForecasted","Edgar.forecast('EarningsPerShareBasic')")
    .addFormula("NetIncomeSurprise%","Edgar.surprisePercent('NetIncomeLoss')")
    .addFormula("RevenueSurprise%","Edgar.surprisePercent('Revenue')")
    .addFormula("EarningsPerShareSurprise%","Edgar.surprisePercent('EarningsPerShareBasic')")
    .removeParameterNames("Revenues","SalesRevenueNet")



ch.pschatzmann.edgar.reporting.company.CompanyEdgarValuesDB@71b01e8e

In [4]:
values.getTable

We convert the data to Scala as a preparation so that we can create some charts

In [5]:
import scala.collection.JavaConverters._

val scalaValues = values.toList.asScala
    .map(map => map.asScala.toMap)
    .toSeq

scalaValues.size

45

We check if the forecast make any sense

In [6]:
val plot = new SimpleTimePlot {
    data = scalaValues
    columns = Seq("NetIncomeLoss","NetIncomeForecasted")
}


And last but not least we finish with the plot of the calculated Surprises:

In [7]:
val plot = new SimpleTimePlot {
    data = scalaValues
    columns = Seq("NetIncomeSurprise%","RevenueSurprise%","EarningsPerShareSurprise%")
}


In [8]:
import ch.pschatzmann.common.table.TableCalculated

val formulas = values.getTable().getBaseTableOfClass("TableCalculated").asInstanceOf[TableCalculated].getFormulas
formulas.getForecast

ch.pschatzmann.edgar.table.forecast.ForecastQuartersCumulated@71f4a619

## Defining an alternative Forecasting Method
We can change the Forecasting method by setting the Forecast implementation on the Formulas object. This needs to be 
done before the values are renderend. In the following example we use the Arima forecast implementation

In [9]:
import  ch.pschatzmann.edgar.table.forecast.ForecastQuartersCumulatedARIMA

val values1 = new CompanyEdgarValuesDB(new CompanySelection().setTradingSymbol("AAPL"))
    .setAddTime(true)
    .setFilter(new FilterQuarterlyCumulated())
    .setParameterNames("NetIncomeLoss", "Revenues","SalesRevenueNet","EarningsPerShareBasic")
    .addFormula("Revenue","Edgar.coalesce('Revenues', 'SalesRevenueNet')")
    .addFormula("NetIncomeForecasted","Edgar.forecast('NetIncomeLoss')")
    .addFormula("RevenueForecasted","Edgar.forecast('Revenue')")
    .addFormula("EarningsPerShareForecasted","Edgar.forecast('EarningsPerShareBasic')")
    .addFormula("NetIncomeSurprise%","Edgar.surprisePercent('NetIncomeLoss')")
    .addFormula("RevenueSurprise%","Edgar.surprisePercent('Revenue')")
    .addFormula("EarningsPerShareSurprise%","Edgar.surprisePercent('EarningsPerShareBasic')")
    .removeParameterNames("Revenues","SalesRevenueNet")

val formulas1 = values1.getTable().getBaseTableOfClass("TableCalculated").asInstanceOf[TableCalculated].getFormulas
formulas1.setForecast(new ForecastQuartersCumulatedARIMA())

values1.getTable

In [10]:
import scala.collection.JavaConverters._

val scalaValues1 = values1.toList.asScala
    .map(map => map.asScala.toMap)
    .toSeq

val plot = new SimpleTimePlot {
    data = scalaValues1
    columns = Seq("NetIncomeSurprise%","RevenueSurprise%","EarningsPerShareSurprise%")
}
